In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_matching/ant/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
from transformers import BertTokenizer, TFBertLMHeadModel
import os
import json
import time
import logging
import pprint
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import random

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Enabled: True


In [4]:
params = {
  'pretrain_path': 'bert-base-chinese',
  'train_path': '../data/train.json',
  'test_path': '../data/dev.json',
  'batch_size': 16,
  'max_len': 130,
  'buffer_size': 34334,
  'init_lr': 1e-5,
  'max_lr': 3e-5,
  'n_epochs': 4 * 10,
}

tokenizer = BertTokenizer.from_pretrained(params['pretrain_path'],
                                          lowercase = True,
                                          add_special_tokens = True)

In [5]:
# stream data from text files
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = json.loads(line.rstrip())
      text1, text2, label = line['sentence1'], line['sentence2'], line['label']
      if len(text1) + len(text2) + 3 > params['max_len']:
        _max_len = (params['max_len'] - 3) // 2
        text1 = text1[:_max_len]
        text2 = text2[:_max_len]
      text1 = list(text1)
      text2 = list(text2)
      text = ['[CLS]'] + text1 + ['[SEP]'] + text2 + ['[SEP]']
      seg = [0] + [0] * len(text1) + [0] + [1] * len(text2) + [1]
      text = tokenizer.convert_tokens_to_ids(text)
      
      noises = []
      labels_mask = []
      for idx in text:
        if (random.random() <= 0.15) and (idx != 101) and (idx != 102) and (idx != 100):
          dice = random.random()
          if dice <= 0.8:
            noises.append(103)
          elif dice <= 0.9:
            noises.append(idx)
          else:
            noises.append(random.randint(0, 21127))
          labels_mask.append(1)
        else:
          noises.append(idx)
          labels_mask.append(0)

      yield (noises, seg), (text, labels_mask)


def dataset(is_training, params):
  _shapes = (([None], [None]), ([None], [None]))
  _types = ((tf.int32, tf.int32), (tf.int32, tf.int32))
  _pads = ((0, 0), (0, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [6]:
# input stream ids check
(text, seg), (labels, labels_mask) = next(data_generator(params['train_path'], params))
print(text)
print(seg)
print(labels)
print(labels_mask)

Reading ../data/train.json
[101, 6010, 6009, 955, 1446, 5023, 7583, 6820, 103, 103, 103, 2940, 2768, 1044, 2622, 1400, 103, 1408, 102, 955, 103, 3300, 1044, 2622, 103, 3309, 6820, 3315, 1408, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[101, 6010, 6009, 955, 1446, 5023, 7583, 6820, 3621, 1377, 809, 2940, 2768, 1044, 2622, 1400, 3315, 1408, 102, 955, 1446, 3300, 1044, 2622, 1168, 3309, 6820, 3315, 1408, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]


In [7]:
model = TFBertLMHeadModel.from_pretrained(params['pretrain_path'],
                                          trainable = True,
                                          return_dict = True)
model.load_weights('../model/bert_further_pretrain.h5')

If you want to use `TFBertLMHeadModel` as a standalone, add `is_decoder=True.`
Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertLMHeadModel: ['nsp___cls']
- This IS expected if you are initializing TFBertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertLMHeadModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertLMHeadModel for predictions without further training.


In [8]:
step_size = 4 * params['buffer_size'] // params['batch_size']
decay_lr = tfa.optimizers.Triangular2CyclicalLearningRate(
  initial_learning_rate = params['init_lr'],
  maximal_learning_rate = params['max_lr'],
  step_size = step_size,)
optim = tf.optimizers.Adam(params['init_lr'])
global_step = 0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

# Baseline Accuracy
m = tf.keras.metrics.Accuracy()
for ((text, seg), (labels, labels_mask)) in dataset(is_training=False, params=params):
  logits = model([text, tf.sign(text), seg], training=False).logits
  m.update_state(
    y_true = labels,
    y_pred = tf.argmax(logits, -1),
    sample_weight = labels_mask,)
best_acc = m.result().numpy()
logger.info("Baseline Accuracy: {:.3f}".format(best_acc))

for _ in range(params['n_epochs']):
  # Training
  for ((text, seg), (labels, labels_mask)) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model([text, tf.sign(text), seg], training=True).logits
      loss = tf.compat.v1.losses.softmax_cross_entropy(
        onehot_labels = tf.one_hot(labels, 21128),
        logits = logits,
        weights = tf.cast(labels_mask, tf.float32),
        label_smoothing = .2,)
    
    trainable_vars = [v for v in model.trainable_variables if 'pooler' not in v.name]
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, trainable_vars)
    grads, _ = tf.clip_by_global_norm(grads, 5.)
    optim.apply_gradients(zip(grads, trainable_vars))
    
    if global_step % 100 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
        global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
  # Evaluation
  m = tf.keras.metrics.Accuracy()

  for ((text, seg), (labels, labels_mask)) in dataset(is_training=False, params=params):
    logits = model([text, tf.sign(text), seg], training=False).logits
    m.update_state(
      y_true = labels,
      y_pred = tf.argmax(logits, -1),
      sample_weight = labels_mask,)

  acc = m.result().numpy()
  logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))

  if acc > best_acc:
    best_acc = acc
    model.save_weights('../model/bert_further_pretrain.h5', save_format='h5')

  logger.info("Best Accuracy: {:.3f}".format(best_acc))

Reading ../data/dev.json
INFO:tensorflow:Baseline Accuracy: 0.851
Reading ../data/train.json
INFO:tensorflow:Step 0 | Loss: 3.0320 | Spent: 42.8 secs | LR: 0.000010
INFO:tensorflow:Step 100 | Loss: 2.7848 | Spent: 33.9 secs | LR: 0.000010
INFO:tensorflow:Step 200 | Loss: 2.7437 | Spent: 33.9 secs | LR: 0.000010
INFO:tensorflow:Step 300 | Loss: 3.0670 | Spent: 34.4 secs | LR: 0.000011
INFO:tensorflow:Step 400 | Loss: 2.9599 | Spent: 33.9 secs | LR: 0.000011
INFO:tensorflow:Step 500 | Loss: 3.1526 | Spent: 33.6 secs | LR: 0.000011
INFO:tensorflow:Step 600 | Loss: 3.0552 | Spent: 33.9 secs | LR: 0.000011
INFO:tensorflow:Step 700 | Loss: 3.0144 | Spent: 34.5 secs | LR: 0.000012
INFO:tensorflow:Step 800 | Loss: 2.8411 | Spent: 33.7 secs | LR: 0.000012
INFO:tensorflow:Step 900 | Loss: 2.7428 | Spent: 34.5 secs | LR: 0.000012
INFO:tensorflow:Step 1000 | Loss: 2.9510 | Spent: 33.9 secs | LR: 0.000012
INFO:tensorflow:Step 1100 | Loss: 2.6413 | Spent: 35.1 secs | LR: 0.000013
INFO:tensorflow:Ste

KeyboardInterrupt: ignored

In [9]:
model.load_weights('../model/bert_further_pretrain.h5')
m = tf.keras.metrics.Accuracy()
for ((text, seg), (labels, labels_mask)) in dataset(is_training=False, params=params):
  logits = model([text, tf.sign(text), seg], training=False).logits
  m.update_state(
    y_true = labels,
    y_pred = tf.argmax(logits, -1),
    sample_weight = labels_mask,)
best_acc = m.result().numpy()
print("MLM Accuracy: {:.3f}".format(best_acc))
print(model.weights[5]) # for later check if the weight is correctly transferred to other task

Reading ../data/dev.json
MLM Accuracy: 0.856
<tf.Variable 'tf_bert_lm_head_model/bert/encoder/layer_._0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32, numpy=
array([[ 0.12166668,  0.00772147,  0.01382446, ..., -0.05134514,
         0.01727283,  0.01975559],
       [-0.01919131, -0.01730992,  0.00995409, ...,  0.01526493,
        -0.0815515 , -0.02229767],
       [ 0.00750611, -0.00214197,  0.02398042, ...,  0.00636234,
        -0.06394118, -0.00126247],
       ...,
       [ 0.00831017, -0.00127873,  0.11854228, ...,  0.05074448,
        -0.0348739 ,  0.09707301],
       [ 0.01920506,  0.04018742, -0.03493238, ..., -0.00172372,
         0.04089048, -0.0429044 ],
       [-0.07598049,  0.02195707, -0.01592555, ...,  0.06175591,
        -0.02131575, -0.00702451]], dtype=float32)>
